# American-Style Derivatives


In [1]:
import tensorflow as tf
from tensorflow import keras 
import numpy as np
from scipy.stats import norm

# model parameters
d = 5
T = 1.
N = 3
r = .05
delta_i = .1
sigma_i = .2
s_0 = 100.
K = 100.
batch_size = 8192
training_steps = 5000
mc_runs = 500
mc_runs_ub = 16
batch_size_ub = 128
nested_batch_size = 2048

## Payoff and sample paths

In [2]:
# payoff
@tf.function
def payoff(t, S_t, K):
    return tf.exp(-r * t) * tf.maximum(tf.reduce_max(S_t, axis=1, keepdims=True) - K, 0.)

# generation of sample paths
@tf.function
def sample_paths(T, N, d, r, delta_i, sigma_i, s_0, K, batch_size, dtype=tf.float32):
    dW = tf.random.normal(shape=(batch_size, d, N), stddev=np.sqrt(T / N), dtype=dtype)
    W_t = tf.cumsum(dW, axis=2)
    t = tf.linspace(start=T / N, stop=T, num=N)
    S_t = tf.exp((r - delta_i - sigma_i ** 2 / 2.) * t + sigma_i * W_t) * s_0
    return S_t, payoff(t, S_t, K), t

In [3]:
S_t, payoff, t = sample_paths(T, N, d, r, delta_i, sigma_i, s_0, K, batch_size)

In [4]:
S_t.shape

TensorShape([8192, 5, 3])

## Neural networks

In [5]:
# generates N - 1 single neural networks
def dos_model_single(d, N, batch_size):
    def neural_network(x):
        x = keras.layers.BatchNormalization(axis=1, momentum=0.9)(x)
        x = keras.layers.Dense(d + 40, use_bias=False)(x)
        x = keras.layers.BatchNormalization(axis=1, momentum=0.9)(x)
        x = tf.nn.relu(x)
        x = keras.layers.Dense(d + 40, use_bias=False)(x)
        x = keras.layers.BatchNormalization(axis=1, momentum=0.9)(x)
        x = tf.nn.relu(x)
        x = keras.layers.Dense(1, use_bias=False)(x)
        x = keras.layers.BatchNormalization(axis=1, momentum=0.9)(x)
        return x

    inputs = keras.Input((d + 1, N-1), batch_size=batch_size)
    nets = []

    for i in range(N-1):
        nets.append(neural_network(inputs[:, :, i]))

    return keras.Model(inputs=inputs, outputs=tf.stack(nets, axis=2), name="dos_model")


# a class for the combined layers
class CombinedLayer(keras.layers.Layer):
    def __init__(self, units=32):
        super(CombinedLayer, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[2], input_shape[1], self.units),
            initializer='glorot_uniform',
            trainable=True,
        )

    def call(self, inputs):
        return tf.transpose(tf.matmul(tf.transpose(inputs, [2, 0, 1]), self.w), [1, 2, 0])


# generates N - 1 combined neural networks
def dos_model_combined(d, N, batch_size):

    def batch_norm(x):
        shape = x.get_shape().as_list()
        x = tf.reshape(x, (-1, shape[1] * shape[2]))
        x = keras.layers.BatchNormalization(axis=1, momentum=0.9)(x)
        x = tf.reshape(x, (-1, shape[1], shape[2]))
        return x

    inputs = keras.Input((d + 1, N-1), batch_size=batch_size)
    x = batch_norm(inputs)
    x = CombinedLayer(d + 40)(x)
    x = batch_norm(x)
    x = tf.nn.relu(x)
    x = CombinedLayer(d + 40)(x)
    x = batch_norm(x)
    x = tf.nn.relu(x)
    x = CombinedLayer(1)(x)
    outputs = batch_norm(x)
  
    return keras.Model(inputs=inputs, outputs=outputs, name="dos_model")



## Loss function

In [6]:
# loss function
@tf.function
def calculate_loss(g_t, nets):
    loss = 0.
    g_tau = g_t[:, :, N-1]

    for k in range(N-2, -1, -1):
        net_k = nets[:, :, k]
        f_n = tf.sigmoid(net_k)
        loss -= (g_t[:, :, k] * f_n + g_tau * (1. - f_n))
        g_tau = tf.where(tf.stop_gradient(net_k) > 0., g_t[:, :, k], g_tau)

    return tf.reduce_mean(loss), g_tau

## Training

In [7]:
# generate the model, i.e. the neural networks
model = dos_model_combined(d, N, batch_size)
optimizer=keras.optimizers.Adam(0.01, epsilon=0.1)

# one training step
@tf.function
def train_step(S_t, g_t):
    sample = tf.concat([S_t[:, :, :N-1], g_t[:, :, :N-1]], axis=1)
    with tf.GradientTape() as tape:
        nets = model(sample, training=True)
        loss, g_tau = calculate_loss(g_t, nets)
    optimizer.minimize(loss, model.trainable_weights, tape=tape)
    return loss, tf.reduce_mean(g_tau)


# the training loop
for step in range(training_steps):
    S_t, g_t, _ = sample_paths(T, N, d, r, delta_i, sigma_i, s_0, K, batch_size)
    loss, px = train_step(S_t, g_t)

    if step % 250 == 0:
        print("Training loss (for one batch) at step %d: %.4f" % (step, float(loss)))
        print("Lower bound (for one batch) at step %d: %.4f" % (step, float(px)))
 
    if (step + 1) % 1500 == 0:
        optimizer.learning_rate = optimizer.learning_rate / 10.



Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `

TypeError: minimize() got an unexpected keyword argument 'tape'

## Lower bound

In [ ]:
# a helper function for the calculation of g_tau
@tf.function
def get_g_tau(g_t, nets):
    g_tau = g_t[:, :, N-1]

    for k in range(N-2, -1, -1):
        g_tau = tf.where(tf.stop_gradient(nets[:, :, k]) > 0., g_t[:, :, k], g_tau)

    return g_tau


# monte carlo for the lower bound
p_m_l, p_v_l = [], []
for i in range(mc_runs):
    S_t, g_t, _ = sample_paths(T, N, d, r, delta_i, sigma_i, s_0, K, batch_size)
    sample = tf.concat([S_t[:, :, :N-1], g_t[:, :, :N-1]], axis=1)
    nets = model(sample, training=False)
    g_tau = get_g_tau(g_t, nets)
    p_m, p_v = tf.nn.moments(g_tau, axes=[0])
    p_m_l.append(p_m)
    p_v_l.append(p_v)


# calculation of mean and CI
p_m_l = np.array(p_m_l)
p_m = np.mean(p_m_l)
p_v = np.sum(np.array(p_v_l)) / mc_runs + np.var(p_m_l)
tmp = norm.ppf(0.975) * np.sqrt(p_v / (mc_runs * batch_size - 1.))

g_0 = tf.squeeze(payoff(0., tf.ones((1, 5)) * s_0, K))

# test if we should stop at 0  
if (g_0 > p_m):
    p_m = g_0
    tmp = 0.

print("Lower bound: %.4f" % (p_m, ))
print("95%% CI [ %.4f, %.4f ]" % (p_m - tmp, p_m + tmp))

lower_bound = p_m

## Upper bound

In [8]:
# simulation of nested sample path for the upper bound
@tf.function
def nested_sample_paths(T, N, d, r, delta_i, sigma_i, s_0, K, batch_size, nested_batch_size, dtype=tf.float32):
    S_t, g_t, t = sample_paths(T, N, d, r, delta_i, sigma_i, s_0, K, batch_size)

    S_t_is, g_t_is = [], []
    dW = tf.random.normal(shape=(batch_size, nested_batch_size, d, N), stddev=np.sqrt(T / N), dtype=dtype)
    W_t = tf.cumsum(dW, axis=3)
    
    for k in range(N):
        if k > 0:
            S_t_0 = tf.expand_dims(tf.expand_dims(S_t[:, :, k - 1], axis=1), axis=3)
            S_t_i = tf.concat([tf.expand_dims(S_t[:, :, :k], axis=1) * tf.ones([1, nested_batch_size, 1, 1]),
                             tf.exp((r - delta_i - sigma_i ** 2 / 2.) * t[:N - k] + sigma_i * W_t[:, :, :, :N - k]) * S_t_0],
                            axis=3)
        else:
            S_t_i = tf.exp((r - delta_i - sigma_i ** 2 / 2.) * t + sigma_i * W_t) * s_0
        S_t_i = tf.reshape(S_t_i, [batch_size * nested_batch_size, d, N])
        g_t_i = payoff(t, S_t_i, K)
        S_t_is.append(S_t_i)
        g_t_is.append(g_t_i)
    return S_t_is, g_t_is, S_t, g_t


# calculation of the upper bound
p_m_l, p_v_l = [], []
for i in range(mc_runs_ub):
    S_t_is, g_t_is, S_t, g_t = nested_sample_paths(T, N, d, r, delta_i, sigma_i, s_0, K, batch_size_ub, nested_batch_size)

    q = []

    for k in range(N):
    S_t_i, g_t_i = S_t_is[k], g_t_is[k]
    nested_nets = model(tf.concat([S_t_i[:, :, :N-1], g_t_i[:, :, :N-1]], axis=1), training=False)
    g_tau = g_t_i[:, :, N - 1]
    for j in range(N - 1, k, -1):
        g_tau = tf.where(nested_nets[:, :, j - 1] > 0., g_t_i[:, :, j - 1], g_tau)

    q.append(tf.reduce_mean(tf.reshape(g_tau, [batch_size_ub, nested_batch_size, 1]), axis=1))

    nets = model(tf.concat([S_t[:, :, :N-1], g_t[:, :, :N-1]], axis=1), training=False)
    d_m_n = [g_t[:, :, N - 1] - q[N - 1]]

    for k in range(N - 2, -1, -1):
        f_n = tf.cast(nets[:, :, k] > 0., dtype=tf.float32)
        d_m_n.append((g_t[:, :, k] * f_n + q[k + 1] * (1. - f_n)) - q[k])

    m = tf.cumsum(tf.stack(list(reversed(d_m_n)), axis=2), axis=2)

    p_m, p_v = tf.nn.moments(tf.reduce_max(g_t - m, axis=2), axes=[0])
    p_m_l.append(p_m)
    p_v_l.append(p_v)


# calculation of mean and CI
p_m_l = np.array(p_m_l)
p_m = np.mean(p_m_l)
p_v = np.sum(np.array(p_v_l)) / mc_runs_ub + np.var(p_m_l)
tmp = norm.ppf(0.975) * np.sqrt(p_v / (mc_runs_ub * batch_size_ub - 1.))

print("Upper bound: %.4f" % (p_m, ))
print("95%% CI [ %.4f, %.4f ]" % (p_m - tmp, p_m + tmp))

upper_bound = p_m


IndentationError: expected an indented block (<ipython-input-8-bd4de4c85c33>, line 33)